<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion for Decisions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.5"
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.5 Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> 📜 "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

// Display the result of the chat completion request as Markdown
chatResult.Content.DisplayAs("text/markdown");

Here are five widely used decision frameworks that can help improve the quality of decisions:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**  
Developed by military strategist John Boyd, the OODA Loop is a cyclical framework for making decisions in dynamic and fast-changing environments.  
- **Steps**:  
  - **Observe**: Gather information about the situation.  
  - **Orient**: Analyze the information and contextualize it based on prior knowledge and current circumstances.  
  - **Decide**: Choose the best course of action.  
  - **Act**: Implement the decision and monitor its outcomes.  
- **Use Case**: Ideal for situations requiring rapid decision-making, such as business strategy or crisis management.

---

### 2. **The Eisenhower Matrix (Urgent vs. Important)**  
This framework helps prioritize tasks and decisions based on urgency and importance.  
- **Steps**:  
  - Divide tasks into four quadrants:  
    1. **Urgent and Important**: Do immediately.  
    2. **Important but Not Urgent**: Schedule for later.  
    3. **Urgent but Not Important**: Delegate to others.  
    4. **Neither Urgent nor Important**: Eliminate or ignore.  
- **Use Case**: Useful for time management and prioritizing decisions in personal or professional contexts.

---

### 3. **The Six Thinking Hats**  
Developed by Edward de Bono, this framework encourages exploring decisions from multiple perspectives to ensure balanced thinking.  
- **Steps**:  
  - Use six metaphorical "hats," each representing a different mode of thinking:  
    - **White Hat**: Focus on facts and data.  
    - **Red Hat**: Consider emotions and intuition.  
    - **Black Hat**: Identify risks and potential problems.  
    - **Yellow Hat**: Highlight benefits and optimism.  
    - **Green Hat**: Encourage creativity and new ideas.  
    - **Blue Hat**: Manage the decision-making process.  
- **Use Case**: Effective for group decision-making and brainstorming sessions.

---

### 4. **The DECIDE Model**  
This structured framework helps systematically evaluate options and make informed decisions.  
- **Steps**:  
  - **D**efine the problem.  
  - **E**stablish criteria for success.  
  - **C**onsider all possible alternatives.  
  - **I**dentify the best option.  
  - **D**evelop and implement a plan.  
  - **E**valuate the decision and its outcomes.  
- **Use Case**: Useful for complex decisions requiring thorough analysis.

---

### 5. **The Pros and Cons List**  
A simple yet effective framework for weighing the advantages and disadvantages of a decision.  
- **Steps**:  
  - Create two columns: one for "Pros" and one for "Cons."  
  - List all potential benefits and drawbacks of each option.  
  - Evaluate the relative importance of each item and make a decision based on the balance.  
- **Use Case**: Ideal for personal decisions or situations with limited complexity.

---

Each of these frameworks can be adapted to suit different contexts, helping individuals and teams make more thoughtful, informed, and effective decisions.

### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt, openAIPromptExecutionSettings))
{
    Console.Write(content);
}

Here are five widely recognized decision frameworks that can help improve the quality of decisions:

---

### 1. **The OODA Loop (Observe, Orient, Decide, Act)**  
Developed by military strategist John Boyd, the OODA Loop is a cyclical framework designed for rapid decision-making in dynamic environments. It emphasizes adaptability and continuous learning.  
- **Steps**:  
  - **Observe**: Gather information about the situation.  
  - **Orient**: Analyze the information and assess your position relative to the environment.  
  - **Decide**: Choose the best course of action based on your analysis.  
  - **Act**: Implement the decision and monitor the results.  
- **Use Case**: Ideal for situations requiring quick responses, such as business strategy, crisis management, or competitive environments.

---

### 2. **The Eisenhower Matrix (Urgent vs. Important)**  
This framework helps prioritize tasks and decisions based on urgency and importance. It is named after President Dwight D. Eisenh

### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [6]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

// Note: The chat completion service can also be used with a chat history object, which allows for a more conversational context.
var chatResponse = string.Empty;

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

Here are five decision frameworks that are widely used to improve the quality of decisions. Each framework is structured systematically to help assess options, reduce biases, and promote critical thinking:

---

### **1. The Cynefin Framework**
The Cynefin Framework is a sense-making tool typically used to identify the nature of a decision context and guide decision-making processes accordingly. The framework has five domains:

- **Simple (Clear):** Decisions in this domain have clear cause-and-effect relationships. Follow best practices.
  * **Action:** Categorize, assess, and act.

- **Complicated:** Problems in this domain require expert analysis and multiple options. Cause and effect exist, but they’re not immediately evident.
  * **Action:** Analyze options, choose the best one.

- **Complex:** In complex situations, outcomes are unpredictable, and cause and effect can only be understood in hindsight.
  * **Action:** Experiment and learn through iteration.

- **Chaotic:** These situations demand rapid action to stabilize crises.
  * **Action:** Act decisively to mitigate chaos, then learn and adapt.

- **Disordered:** When true categorization isn’t possible, more analysis is needed to determine the situation type.

### **Use Case:** Helps frame situations to determine the appropriate decision-making strategy.

---

### **2. Six Thinking Hats**
Created by Edward de Bono, this framework encourages multi-faceted decision-making by dividing perspectives into six “hats” or modes of thinking:

- **White Hat:** Focuses on data and facts.
- **Red Hat:** Explores intuition, emotions, and gut feelings.
- **Yellow Hat:** Emphasizes optimism, benefits, and the positive outlook of decisions.
- **Black Hat:** Looks at risks, potential downsides, and critical thinking.
- **Green Hat:** Encourages creativity, innovation, and new ideas.
- **Blue Hat:** Oversees the process and ensures structure.

### **Use Case:** Useful for group decision-making and encouraging balanced, holistic thinking.

---

### **3. OODA Loop**
The OODA Loop (Observe, Orient, Decide, Act) is a military-inspired framework developed by Colonel John Boyd. It emphasizes agility and iterative decision-making:

1. **Observe:** Collect and analyze relevant information about your environment and situation.
2. **Orient:** Interpret observations, adjust mental models, and align to contextual realities.
3. **Decide:** Choose the best course of action based on your orientation.
4. **Act:** Implement the decision, then repeat the loop if needed.

### **Use Case:** Particularly beneficial for dynamic scenarios that evolve quickly, such as competitive or uncertain environments.

---

### **4. Decision Matrix Analysis (Weighted Scoring)**
A structured approach for evaluating options with multiple criteria. The framework involves assigning weights to criteria based on their importance and scoring options accordingly.

1. Define decision criteria.
2. Assign weights to each criterion (e.g., on a scale of 1–10).
3. List possible options.
4. Score each option for each criterion, then multiply by the weights.
5. Select the option with the highest total score.

### **Use Case:** Effective for rational, data-driven decision-making, particularly when comparing complex alternatives.

---

### **5. Eisenhower Matrix**
This time-management framework focuses on prioritizing tasks but can be adapted for decision-making in general. It categorizes actions based on urgency and importance:

1. **Important and Urgent:** Do immediately.
2. **Important but Not Urgent:** Plan these actions.
3. **Not Important but Urgent:** Delegate where possible.
4. **Not Important and Not Urgent:** Eliminate or deprioritize.

### **Use Case:** Useful for prioritizing actions in decision-making contexts, especially under time constraints.

---

### **Final Thoughts:**
Each framework offers different advantages based on the nature of the decision, the complexity of the situation, and the decision-maker's objectives. Picking the right tool depends on whether you need adaptability (OODA Loop), analytical structure (Decision Matrix), empathy and balance (Six Thinking Hats), or contextual awareness (Cynefin Framework). Consider integrating multiple frameworks where possible to account for varied factors.

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> 📜 "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community?  
Think carefully step by step about what decision frameworks are needed to answer the query.  
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

To determine which decision framework is best suited for the military intelligence community, it’s important to analyze the characteristics of the environment, the expectations of intelligence work, and the qualities of effective decision-making within such contexts. Let’s evaluate step-by-step which framework is most appropriate.

---

### Step 1: **Understand the Military Intelligence Context**
Military intelligence operates in environments that are:
- **Dynamic:** Situations can change quickly, often unpredictably.
- **Complex:** Decisions involve analyzing large amounts of data, understanding adversary behavior, and assessing risks to national security.
- **Uncertain:** There is often incomplete or ambiguous information available.
- **Time-sensitive:** Delays can have significant operational consequences and may undermine mission success.
- **Iterative:** Continuous analysis and reevaluation of situations are critical as new information becomes available.

The decision-making framework will likely need to address **uncertainty**, **agility**, and **iteration**, while minimizing biases and maximizing operational efficiency.

---

### Step 2: **Evaluate the Five Frameworks**
Let’s examine each framework against the criteria established above:

#### 1. **Cynefin Framework**
The Cynefin Framework recognizes the complexity and dynamic nature of environments by categorizing them into simple, complicated, complex, and chaotic domains. Given the diverse nature of military intelligence situations (e.g., strategy formulation, crisis management, and battlefield tactics), this framework provides a high-level structure for understanding the type of decision-making required. However, it lacks specific procedural guidance for real-time, iterative intelligence analysis.

#### 2. **Six Thinking Hats**
While the Six Thinking Hats framework encourages diverse perspectives and balances emotions, creativity, and logic, it’s better suited to group deliberations and brainstorming sessions. The intelligence community requires decision-making frameworks that emphasize urgency, agility, and iterative processes—which are less emphasized in the Six Thinking Hats approach.

#### 3. **OODA Loop**
The OODA Loop (Observe, Orient, Decide, Act) is specifically designed for dynamic, fast-changing environments that require iterative decisions based on evolving information. Intelligence gathering aligns naturally with the **Observe** phase, where data is collected and analyzed. The **Orient** phase allows analysts to contextualize data (e.g., geopolitical factors, adversary motivations). The **Decide** phase enables quick course selection, and the **Act** phase ensures execution of a decision while preparing for the next iteration. This framework is highly effective for military intelligence because it accounts for uncertainty, time sensitivity, and adaptation to new information.

#### 4. **Decision Matrix Analysis**
The Decision Matrix is ideal for structured, analytical reasoning with multiple criteria. However, military intelligence often involves fluid and uncertain situations that demand adaptation rather than static comparisons. While useful for certain technical evaluations (e.g., selecting equipment, planning logistics), the matrix lacks the agility needed to navigate the fast-paced nature of intelligence work.

#### 5. **Eisenhower Matrix**
The Eisenhower Matrix is designed for prioritizing tasks (urgent vs. important) rather than making comprehensive operational decisions. Though prioritization is necessary for intelligence, military operations often deal with overlapping urgency and importance, making this framework less suitable for navigating complex, evolving circumstances.

---

### Step 3: **Reasoning Through Selection**
The framework most suited for the military intelligence community must:
- Enable real-time adaptation to new data and circumstances.
- Function effectively under uncertainty and time pressures.
- Support iterative decision-making as intelligence evolves.

Among the frameworks, the **OODA Loop** aligns the most closely with these requirements. Its focus on agility, iterative thinking, and rapid response makes it ideal for military intelligence scenarios.

---

### Final Recommendation: **OODA Loop**

The OODA Loop (Observe, Orient, Decide, Act) is the single best framework for use within the military intelligence community. This framework supports critical real-time situational awareness and decision-making in dynamic, high-stakes environments, enabling intelligence professionals to adapt to evolving challenges while maintaining operational effectiveness.

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> 📜 "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

Semantic Kernel can maintain state with the ChatHistory object. This allows natural converation between the user and AI to be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [8]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. 

In [9]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

The Fitbit Inspire 3 is an excellent choice for balancing affordability, battery life (up to 10 days), and reliable sleep tracking. It also includes features such as heart rate monitoring, activity tracking, and compatibility with the Fitbit app for deeper insights—all of which should meet your needs well.

Final thoughts:
1. **Comfort and style**: Make sure the band is comfortable for long-term wear, especially since you'll be using sleep tracking at night.
2. **App integration**: Spend a little time exploring the Fitbit app—it has valuable information and tools to maximize your tracker’s benefits for health and fitness.
3. **Charging habits**: Since it has a long battery life, consider developing a routine (e.g., charging while you shower) to avoid interruptions.

Enjoy your Fitbit! It sounds like a great fit for staying on top of battery life and sleep insights!

### Step 6 - Inspect Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

> 📝 Note: You can look at Chat History as a type of gathered intelligence in the Decision Intelligence framework. The interactions between users, AI models, AI processes and AI agents are all critical to use in decision-making processes. It is highly recommended to persist these chat history objects (agent threads) especially during the decision optimization steps.  

In [30]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

// Change this to a string builder and show as markdown
var stringBuilderChatHistory = new StringBuilder();
foreach (var message in chatHistoryDecisionConversation)
{
    // add a new line for each message
    stringBuilderChatHistory.AppendLine($"**{message.Role.ToString().ToUpper()}**:");
    stringBuilderChatHistory.Append($"{message.Content.Replace("#", string.Empty)}");
    stringBuilderChatHistory.AppendLine("\n");
}

// Display the chat history as Markdown
stringBuilderChatHistory.ToString().DisplayAs("text/markdown");

Number of chat interactions: 10


**SYSTEM**:
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

**USER**:
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

**ASSISTANT**:
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

**USER**:
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?

**ASSISTANT**:
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.

**USER**:
Are there any popular models you would recommend for that balance of battery life and sleep tracking?

**ASSISTANT**:
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.

**USER**:
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?

**ASSISTANT**:
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 

**USER**:
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts on my decision? 



By persisting the Decision Intelligence chats and making them available for inspection, can further help the decision-making process. For example you can load the Chat History summarize it or even have AI recommend optimizations for it based on the conversation. 

> 📝 Note: The process below is simplified to illustrate the power of Gathering Intelligence with user <--> AI interactions. More advanced best practices will be introduced in further sections. 

In [33]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
chatResponse = chatContent.Content;
chatResponse.DisplayAs("text/markdown");

// Optimize the chat history for any future decision interactions
var userPromptOptimizeTheDecisionChat = """
Based on the chat conversation so far...
Provide some recommendations on how the decision interactions could be optimized for decision-making.
What are some considerations for the AI to help the use make even better decisions?
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptOptimizeTheDecisionChat);

var chatContentOptimization = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
var chatResponseOptimization = chatContentOptimization.Content;
chatResponseOptimization.DisplayAs("text/markdown");

You considered buying a fitness tracker and identified key priorities like battery life and sleep tracking. After exploring typical battery ranges (2-14 days) and features, I recommended models like the Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, and Xiaomi Mi Band 7 for their balance of these features. You also compared the Apple Watch, which offers advanced features but shorter battery life. Ultimately, you decided that the Fitbit Inspire 3 aligns best with your budget and needs.

To optimize decision-making in interactions like these, the AI could refine its approach in the following ways:

### 1. **Structure the Decision Process More Explicitly**
   - Break down the decision into stages (e.g., identifying priorities → exploring options → comparing trade-offs → confirming the decision).
   - Use a decision matrix or criteria checklist to help the user assess choices more objectively.

### 2. **Provide Tailored Comparisons**
   - List pros and cons for each option based purely on the user's stated needs (e.g., battery life and sleep tracking).
   - Highlight edge cases or limitations, such as charging constraints or variances in data accuracy.

### 3. **Introduce Additional Decision Factors**
   - Suggest considering warranty, durability, water resistance, and long-term compatibility with the user's other devices.
   - Incorporate the ecosystem of the device (e.g., ease-of-use of the app or long-term software updates).

### 4. **Expand Contextual Insights**
   - Offer potential future needs the user might not currently anticipate, like stress monitoring or compatibility with new health-tech trends.
   - Highlight ongoing benefits (e.g., community features within Fitbit or motivation provided by competitions).

### 5. **Guide Post-Decision Planning**
   - Provide tips on next steps after choosing the tracker (e.g., explore app tutorials, set fitness goals, or optimize sleep tracking settings).
   - Briefly discuss user feedback/reviews to reinforce the choice made.

### 6. **Use More Visual Tools**
   - Present data visually via tables, charts, or side-by-side comparisons to help the user more clearly evaluate options in one glance.

### 7. **Address Cognitive Biases**
   - Mitigate decision fatigue by summarizing a few top choices based on stated priorities.
   - Help the user question assumptions about brands or features they might value excessively (e.g., pointing out trade-offs of the Apple Watch).

By integrating these aspects, decision-making becomes more thorough, systematic, and engaging, enabling the user to confidently choose from options while prioritizing their needs.